```
LDFLAGS="-L/usr/lib/x86_64-linux-gnu" ./configure --with-ensurepip --prefix=/opt/python-3.5.2 --enable-shared
make
sudo make install
export LD_LIBRARY_PATH=/opt/python-3.5.2/lib:$LD_LIBRARY_PAT
```

```
sudo apt-get install libopenblas-base
sudo apt-get install libopenblas-dev
sudo apt-get install python-node
pip install Theano
```

In [2]:
import theano
from theano import tensor as T

In [3]:
x1 = T.scalar()

In [4]:
w1 = T.scalar()
w0 = T.scalar()

In [5]:
z1 = w1 * x1 + w0

In [6]:
net_input = theano.function(inputs=[w1, x1, w0], outputs=z1)

In [7]:
print('Net input: %.2f' % net_input(2.0, 1.0, 0.5))

Net input: 2.50


In [8]:
theano.config.floatX = 'float32'

In [9]:
theano.config.device

'cpu'

In [10]:
import numpy as np

In [11]:
x = T.fmatrix(name='x')

In [12]:
x_sum = T.sum(x, axis=0)

In [13]:
calc_sum = theano.function(inputs=[x], outputs=x_sum)

In [14]:
ary = [[1, 2, 3], [1, 2, 3]]

In [15]:
print('Column sum:', calc_sum(ary))

Column sum: [ 2.  4.  6.]


In [16]:
ary = np.array([[1, 2, 3], [1, 2, 3]], dtype=theano.config.floatX)

In [17]:
print('Column sum:', calc_sum(ary))

Column sum: [ 2.  4.  6.]


In [18]:
x = T.fmatrix('x')
w = theano.shared(np.array([[0.0, 0.0, 0.0]], dtype=theano.config.floatX))
z = x.dot(w.T)

In [19]:
update = [[w, w + 1.0]]

In [20]:
net_input = theano.function(inputs=[x], updates=update, outputs=z)

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [21]:
data = np.array([[1, 2, 3]], dtype=theano.config.floatX)

In [23]:
for i in range(5):
    print('z%d:' % i, net_input(data))

z0: [[ 0.]]
z1: [[ 6.]]
z2: [[ 12.]]
z3: [[ 18.]]
z4: [[ 24.]]


# Keras

In [28]:
import os
import struct


def load_mnist(path, kind='train'):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        
    return images, labels


In [29]:
X_train, y_train = load_mnist('mnist', kind='train')

In [31]:
X_test, y_test = load_mnist('mnist', kind='t10k')

In [32]:
import theano
theano.config.fleatX = 'float32'

In [33]:
X_train = X_train.astype(theano.config.floatX)

In [34]:
X_test = X_test.astype(theano.config.floatX)

In [36]:
from keras.utils import np_utils

Using TensorFlow backend.


In [37]:
y_train_ohe = np_utils.to_categorical(y_train)

In [38]:
y_train_ohe[:3]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [39]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

In [40]:
np.random.seed(1)

In [59]:
model = Sequential()

In [60]:
model.add(Dense(input_dim=X_train.shape[1], output_dim=50, init='uniform', activation='tanh'))

In [61]:
model.add(Dense(input_dim=50, output_dim=50, init='uniform', activation='tanh'))

In [62]:
model.add(Dense(input_dim=50, output_dim=y_train_ohe.shape[1], init='uniform', activation='softmax'))

In [63]:
y_train_ohe.shape[1]  # The number of class labels

10

In [64]:
X_train.shape[1]  # The number of features

784

In [65]:
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)

In [66]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [67]:
model.fit(X_train, y_train_ohe, nb_epoch=50, batch_size=3000, verbose=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 1s - loss: 2.2945 - acc: 0.1495 - val_loss: 2.2876 - val_acc: 0.1893
Epoch 2/50
54000/54000 [==============================] - 0s - loss: 2.2822 - acc: 0.2286 - val_loss: 2.2750 - val_acc: 0.2782
Epoch 3/50
54000/54000 [==============================] - 0s - loss: 2.2696 - acc: 0.3167 - val_loss: 2.2615 - val_acc: 0.3670
Epoch 4/50
54000/54000 [==============================] - 0s - loss: 2.2559 - acc: 0.3888 - val_loss: 2.2466 - val_acc: 0.4348
Epoch 5/50
54000/54000 [==============================] - 1s - loss: 2.2407 - acc: 0.4452 - val_loss: 2.2299 - val_acc: 0.4855
Epoch 6/50
54000/54000 [==============================] - 1s - loss: 2.2234 - acc: 0.4932 - val_loss: 2.2109 - val_acc: 0.5283
Epoch 7/50
54000/54000 [==============================] - 0s - loss: 2.2035 - acc: 0.5261 - val_loss: 2.1891 - val_acc: 0.5585
Epoch 8/50
54000/54000 [==============================] - 0s -

In [68]:
y_train_pred = model.predict_classes(X_train, verbose=0)

In [69]:
print('first 3 predictions: ', y_train_pred[:3])

first 3 predictions:  [3 0 4]


In [70]:
train_acc = np.sum(y_train == y_train_pred, axis=0) / X_train.shape[0]

In [71]:
print('Acc %.2f' % (train_acc * 100))

Acc 84.57


In [72]:
y_test_pred = model.predict_classes(X_test, verbose=0)

In [73]:
test_acc = np.sum(y_test == y_test_pred, axis=0) / X_test.shape[0]

In [74]:
test_acc

0.84419999999999995